In [71]:
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import pandas as pd
import json
from IPython.display import JSON

from binance.spot import Spot 

# handle data
import pyspark.sql as ps
from pyspark.sql.functions import from_unixtime,date_format,from_utc_timestamp
from pyspark.sql.types import DateType
from pyspark.sql import Row

# train data
import pandas as pd
import numpy as np
from datetime import datetime,timedelta

# enviroment
import os
from dotenv import load_dotenv
load_dotenv("../../env/app.env")

True

# 1. Get token details

### Declare api routes

In [40]:
COIN_MARKET_CAP_KEY = os.environ.get("COIN_MARKET_CAP_KEY")
API_URL = 'https://pro-api.coinmarketcap.com'

META_DATA_ROUTE = '/v2/cryptocurrency/info'
LIST_SYMBOLS_ROUTE = '/v1/cryptocurrency/map'

### Get all symbols

In [41]:
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': COIN_MARKET_CAP_KEY,
}
session = Session()
session.headers.update(headers)

In [42]:
response = session.get(API_URL + LIST_SYMBOLS_ROUTE)
data = json.loads(response.text)

In [45]:
JSON(data["data"][:2])

<IPython.core.display.JSON object>

In [93]:
symbols = list(map(lambda info: info["symbol"],data["data"]) ) 

In [47]:
print(len(symbols))
print(symbols[:10])

2124
['ETHBTC', 'LTCBTC', 'BNBBTC', 'NEOBTC', 'QTUMETH', 'EOSETH', 'SNTETH', 'BNTETH', 'BCCBTC', 'GASBTC']


### Fetch detail of each symbol

In [161]:
parameters = {
  'symbol':",".join(symbols[:1060]),
  'aux': "description,tags,date_added"
}

response = session.get(API_URL + META_DATA_ROUTE, params=parameters)
data = json.loads(response.text)

In [162]:
def get_table(data):
    items = list(map(lambda item: item[1][0],data.items()))
    rows = list(map(lambda item: list(map(lambda k_v: k_v[1],item.items())) ,items))
    cols = list(map(lambda k_v: k_v[0],items[0].items()))
    
    return rows,cols

In [163]:
rows, cols = get_table(data["data"])
df = pd.DataFrame(rows, columns=cols)

In [170]:
df

,id,name,symbol,category,description,slug,subreddit,tags,tag-names,tag-groups,date_added,twitter_username,is_hidden,date_launched,contract_address,self_reported_circulating_supply,self_reported_tags,self_reported_market_cap
0,93,42-coin,42,coin,42-coin (42) is a cryptocurrency . Users are a...,42-coin,,"[mineable, hybrid-pow-pos, scrypt, store-of-va...","[Mineable, Hybrid - PoW & PoS, Scrypt, Store O...","[OTHERS, ALGORITHM, ALGORITHM, CATEGORY]",2014-01-14T00:00:00.000Z,42newchain,0,None,[{'contract_address': '0x73cf73c2503154de4dc12...,NaN,None,NaN
1,2837,0xBitcoin,0xBTC,token,0xBitcoin (0xBTC) is a cryptocurrency and oper...,0xbtc,0xbitcoin,"[mineable, arbitrum-ecosytem]","[Mineable, Arbitrum Ecosystem]","[OTHERS, PLATFORM]",2018-06-04T00:00:00.000Z,0xBTCFoundation,0,None,[{'contract_address': '0xb6ed7644c69416d67b522...,8183550.0,None,1.896847e+06
2,2601,1World,1WO,token,1World (1WO) is a cryptocurrency and operates ...,1world,,[algorand-ecosystem],[Algorand Ecosystem],[CATEGORY],2018-03-21T00:00:00.000Z,1World_Online,0,None,[{'contract_address': '0xfdbc1adc26f0f8f8606a5...,NaN,None,NaN
3,1252,2GIVE,2GIVE,coin,2GIVE (2GIVE) is a cryptocurrency . Users are ...,2give,2GIVE,"[mineable, hybrid-pow-pos, scrypt]","[Mineable, Hybrid - PoW & PoS, Scrypt]","[OTHERS, ALGORITHM, ALGORITHM]",2016-05-16T00:00:00.000Z,2GiveCoin,0,None,[],NaN,None,NaN
4,3287,Abulaba,AAA,token,Abulaba (AAA) is a cryptocurrency and operates...,abulaba,,None,None,None,2018-09-14T00:00:00.000Z,AbulabaCapital,0,None,[{'contract_address': '0xd938137e6d96c72e4a608...,NaN,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1041,1896,0x,ZRX,token,0x (ZRX) is a cryptocurrency and operates on t...,0x,0xProject,"[platform, decentralized-exchange-dex-token, d...","[Platform, Decentralized Exchange (DEX) Token,...","[CATEGORY, CATEGORY, CATEGORY, CATEGORY, CATEG...",2017-08-16T00:00:00.000Z,0xproject,0,None,[{'contract_address': '0xe41d2489571d322189246...,NaN,None,NaN
1042,2047,Zeusshield,ZSC,token,Zeusshield (ZSC) is a cryptocurrency and opera...,zeusshield,,"[asset-management, ai-big-data]","[Asset Management, AI & Big Data]","[INDUSTRY, INDUSTRY]",2017-10-13T00:00:00.000Z,Zeusshield,0,None,[{'contract_address': '0x7A41e0517a5ecA4FdbC7F...,NaN,None,NaN
1043,4826,ZUM TOKEN,ZUM,token,ZUM TOKEN (ZUM) is a cryptocurrency launched i...,zum-token,,"[ethereum-ecosystem, bnb-chain]","[Ethereum Ecosystem, BNB Chain]","[PLATFORM, PLATFORM]",2019-10-25T00:00:00.000Z,Zum_Token,0,2019-09-01T00:00:00.000Z,[{'contract_address': '0xe0b9bcd54bf8a730ea5d3...,NaN,None,NaN
1044,1250,Zurcoin,ZUR,coin,Zurcoin (ZUR) is a cryptocurrency . Users are ...,zurcoin,,"[mineable, pow, quark]","[Mineable, PoW, Quark]","[OTHERS, ALGORITHM, ALGORITHM]",2016-05-09T00:00:00.000Z,DmhZur,0,None,[],106031475.0,None,3.481553e+04


# 2. Get average price entire of tokens

### Declare client

In [88]:
BINANCE_API_KEY = os.environ.get("BINANCE_API_KEY")
BINANCE_API_SECRET = os.environ.get("BINANCE_API_SECRET")
client = Spot(key=BINANCE_API_KEY, secret=BINANCE_API_SECRET)
spark = ps.SparkSession.\
        builder.\
        appName("pyspark-notebook").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "4096m").\
        getOrCreate()
spark.sparkContext.setLogLevel("WARN")

22/11/29 15:18:29 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


### Get symbols 

In [89]:
exchanges = client.exchange_info()
symbols = list(map(lambda item: item.get('symbol'), exchanges.get('symbols')))

### Get prices of symbols

In [90]:
data1 = client.ticker_24hr(symbols=symbols[:500])
data2 = client.ticker_24hr(symbols=symbols[500:1000])
data3 = client.ticker_24hr(symbols=symbols[1000:1500])
data4 = client.ticker_24hr(symbols=symbols[1500:2000])
data5 = client.ticker_24hr(symbols=symbols[2000:])

### Gather and store in hdfs

In [91]:
data = [*data1, *data2, *data3, *data4, *data5] 

In [92]:
df = pd.DataFrame(data)
df = spark.createDataFrame(df)
df.show(2)

/usr/local/lib/python3.9/dist-packages/pyspark/sql/pandas/conversion.py:327: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
22/11/29 15:18:51 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources


+------+-----------+------------------+----------------+--------------+----------+-----------+----------+-----------+----------+-----------+----------+----------+----------+---------------+-------------+-------------+-------------+---------+---------+------+
|symbol|priceChange|priceChangePercent|weightedAvgPrice|prevClosePrice| lastPrice|    lastQty|  bidPrice|     bidQty|  askPrice|     askQty| openPrice| highPrice|  lowPrice|         volume|  quoteVolume|     openTime|    closeTime|  firstId|   lastId| count|
+------+-----------+------------------+----------------+--------------+----------+-----------+----------+-----------+----------+-----------+----------+----------+----------+---------------+-------------+-------------+-------------+---------+---------+------+
|ETHBTC| 0.00194700|             2.711|      0.07295540|    0.07182700|0.07377500| 4.38110000|0.07377500| 6.84510000|0.07377600|22.07340000|0.07182800|0.07398200|0.07158300| 83893.54270000|6120.48717935|1669648712438|166973

In [93]:
df.write.option("header",True) \
         .mode("overwrite") \
         .parquet(f"hdfs://hadoop-namenode:9000/crypto/info/symbols.parquet")

In [94]:
spark.stop()